In [1]:
from utils import *
from model import ConformerMSCRED
import torch
from time import time
import numpy as np
import random
import pickle

random.seed(42)
torch.manual_seed(42)

In [2]:
with open('scale_factor.pkl', 'rb') as f:
    max_, min_ = pickle.load(f)

with open('./columns.pkl', 'rb') as f:
    columns = pickle.load(f)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ConformerMSCRED(device=device).to(device)
model.eval()
ckpt_path = '../weights/exp5/mscred_weights_epoch_1_val_loss_5.2967.pt'
model.load_state_dict(torch.load(ckpt_path, map_location='cpu'))

<All keys matched successfully>

In [4]:
def get_result(model, input_data, max_, min_, columns):
    values = []
    for input_dict in input_data:
        value = [input_dict[key] for key in input_dict if key in columns]
        value = list(map(float, value))
        values.append(value)
    values = np.array(values)

    scaled_values, _, _  = MinMaxScaler(values, max_=max_, min_=min_)

    X, y = calculate_signature_matrix_dataset(scaled_values, lags=[10, 30, 60], stride=1, num_timesteps=5)
    X = X.transpose(0, 4, 1, 2, 3)
    y = y.transpose(0, 3, 1, 2)
    X = torch.Tensor(X)

    pred = model(X).detach().cpu().numpy()

    residual_matrix = y - pred
    err = np.sum(residual_matrix**2, axis=1)
    err = np.sum(err, axis=1)
    err = np.sum(err, axis=1) / (pred.shape[1] * pred.shape[2] * pred.shape[3])

    return err[0]

In [5]:
input_dict = {'message': {'TAG_MIN': '2022-01-03 11:22:12', '배정번호': '102410', '건조 1존 OP': '73.6468', '건조 2존 OP': '29.5274', '건조로 온도 1 Zone': '98.7918', '건조로 온도 2 Zone': '99.2075', '세정기': '68.4386', '소입1존 OP': '76.0262', '소입2존 OP': '61.1634', '소입3존 OP': '51.6915', '소입4존 OP': '71.6093', '소입로 CP 값': '0.450235', '소입로 CP 모니터 값': '1.14536e-10', '소입로 온도 1 Zone': '860.351', '소입로 온도 2 Zone': '859.791', '소입로 온도 3 Zone': '859.991', '소입로 온도 4 Zone': '859.731', '솔트 컨베이어 온도 1 Zone': '294.721', '솔트 컨베이어 온도 2 Zone': '272.599', '솔트조 온도 1 Zone': '328.74', '솔트조 온도 2 Zone': '328.808'}}
input_list = [input_dict['message'] for _ in range(64)]

In [6]:
start = time()
mse = get_result(model, input_list, max_, min_, columns)
print(mse)
end = time()
print(f"Time spent:{end - start:0.4f}")

100%|██████████| 5/5 [00:00<00:00, 719.41it/s]

0.22860826661318512
Time spent:0.2113
